# Set up


## Imports

In [1]:
%matplotlib inline

/home/gus/.anaconda/envs/biorepo/lib/python3.5/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
import sys
import os
from pathlib import Path
import math

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context(context="talk", font_scale=1, rc=None)
sns.set_style('whitegrid')

In [4]:
# Load the "autoreload" extension
%load_ext autoreload
%autoreload 2

In [147]:
import pandas as pd
from pandas import IndexSlice as idx
import numpy as np

from collections import OrderedDict

from munch import Munch, munchify
from tqdm import tqdm

In [6]:
import engarde.decorators as ed
import pyparsing as p

In [7]:
# import crunchers.sklearn_helpers.exploration as expl
# import crunchers.statsmodels_helpers.lazy_stats as stats
# import crunchers.pandas_helpers.transformations as xform

In [46]:
import biorep_etl.data.hreg_load_recode as loading
import biorep_etl.data.field_definitions.hreg_redcap_dump as rcd
from biorep_etl.data.parsers import BranchLogicParser

## Constants

### Get info for converting/moving this file to `reports`

In [9]:
# %%javascript
# var kernel = IPython.notebook.kernel;
# var thename = window.document.getElementById("notebook_name").innerHTML;
# var command = "IPYNB_NAME = " + "'"+thename+"'";
# kernel.execute(command);

In [10]:
# rv = Munch()

# rv.rep_id = 'yyy-mm-dd'
# rv.desc = 'blank'
# rv.ipnb = Path('{IPYNB_NAME}.ipynb'.format(IPYNB_NAME=IPYNB_NAME))
# rv.html = '{ipnb}.html'.format(ipnb=rv.ipnb.stem)
# rv.files = '{ipnb}_files'.format(ipnb=rv.ipnb.stem)
# rv.reprt_dir = '../../reports/{IPYNB_NAME}/'.format(IPYNB_NAME=IPYNB_NAME)

### Other Constants

## Paths

In [11]:
data_ = "../data/raw/registry/HarvardInflammatoryB_DATA_2017-01-20_0924.csv"
data_dict_ = "../data/raw/registry/HarvardInflammatoryBowelDiseas_DataDictionary_2017-01-20.csv"

## Functions

## Loading

### Data

In [273]:
d = loading.load_redcap_dump(data_=data_)
d.head()

/home/gus/.anaconda/envs/biorepo/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (0,115,151,162,171,273,424,425,429,432,433,447,448,523,539,664,667,670,673,676,679,682,685) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


visitdate  consent  noconsent  noconsentoth  \
subid redcap_event_name                                                        
1     baseline_visit_arm_1             NaN      NaN        NaN           NaN   
      followup_visit_1_arm_1           NaN      NaN        NaN           NaN   
      followup_visit_2_arm_1           NaN      NaN        NaN           NaN   
      registration_visit_arm_1  2011-05-24      1.0        NaN           NaN   
2     baseline_visit_arm_1             NaN      NaN        NaN           NaN   

                                veoibd  otherstudies  \
subid redcap_event_name                                
1     baseline_visit_arm_1         NaN           NaN   
      followup_visit_1_arm_1       NaN           NaN   
      followup_visit_2_arm_1       NaN           NaN   
      registration_visit_arm_1     NaN           1.0   
2     baseline_visit_arm_1         NaN           NaN   

                                prior_protocol_number___5  \
subid redcap_event_name                                     
1     baseline_visit_arm_1                            NaN   
      followup_visit_1_arm_1                          NaN   
      followup_visit_2_arm_1                          NaN   
      registration_visit_arm_1                        1.0   
2     baseline_visit_arm_1                            NaN   

                                prior_protocol_number___1  \
subid redcap_event_name                                     
1     baseline_visit_arm_1                            NaN   
      followup_visit_1_arm_1                          NaN   
      followup_visit_2_arm_1                          NaN   
      registration_visit_arm_1                        0.0   
2     baseline_visit_arm_1                            NaN   

                                prior_protocol_number___2  \
subid redcap_event_name                                     
1     baseline_visit_arm_1                            NaN   
      followup_visit_1_arm_1                          NaN   
      followup_visit_2_arm_1                          NaN   
      registration_visit_arm_1                        0.0   
2     baseline_visit_arm_1                            NaN   

                                prior_protocol_number___3  \
subid redcap_event_name                                     
1     baseline_visit_arm_1                            NaN   
      followup_visit_1_arm_1                          NaN   
      followup_visit_2_arm_1                          NaN   
      registration_visit_arm_1                        0.0   
2     baseline_visit_arm_1                            NaN   

                                          ...              hosp_k  hosptype_k  \
subid redcap_event_name                   ...                                   
1     baseline_visit_arm_1                ...                 NaN         NaN   
      followup_visit_1_arm_1              ...                 NaN         NaN   
      followup_visit_2_arm_1              ...                 NaN         NaN   
      registration_visit_arm_1            ...                 NaN         NaN   
2     baseline_visit_arm_1                ...                 NaN         NaN   

                                hospibdtype_k  hosp_l  hosptype_l  \
subid redcap_event_name                                             
1     baseline_visit_arm_1                NaN     NaN         NaN   
      followup_visit_1_arm_1              NaN     NaN         NaN   
      followup_visit_2_arm_1              NaN     NaN         NaN   
      registration_visit_arm_1            NaN     NaN         NaN   
2     baseline_visit_arm_1                NaN     NaN         NaN   

                                hospibdtype_l  hosp_m  hosptype_m  \
subid redcap_event_name                                             
1     baseline_visit_arm_1                NaN     NaN         NaN   
      followup_visit_1_arm_1              NaN     NaN         NaN   
      followup_visit_2_arm_1           

### Data Dict

In [274]:
cb = loading.load_data_dict(data_dict_=data_dict_).dropna(axis=1, how='all')
cb.head(2)

,Form Name,Section Header,Field Type,Field Label,"Choices, Calculations, OR Slider Labels",Field Note,Text Validation Type OR Show Slider Number,Text Validation Min,Text Validation Max,Identifier?,Branching Logic (Show field only if...),Required Field?,Custom Alignment,Matrix Group Name
Variable / Field Name,,,,,,,,,,,,,,
subid,registration_visit,NaN,text,Subject ID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
visitdate,registration_visit,Form RV: Registration Visit,text,Date of visit at which patient was consented:,NaN,NaN,date_mdy,NaN,NaN,NaN,NaN,y,NaN,NaN


In [278]:
cb['Field Type'].value_counts()

radio          214
text            80
checkbox        49
dropdown        34
yesno            6
descriptive      5
calc             3
Name: Field Type, dtype: int64

In [279]:
cb[cb['Field Type'] == 'radio'].head()

,Form Name,Section Header,Field Type,Field Label,"Choices, Calculations, OR Slider Labels",Field Note,Text Validation Type OR Show Slider Number,Text Validation Min,Text Validation Max,Identifier?,Branching Logic (Show field only if...),Required Field?,Custom Alignment,Matrix Group Name
Variable / Field Name,,,,,,,,,,,,,,
noconsent,registration_visit,NaN,radio,"If no, provide reason","1, Not interested in Research Studies | 2, Pri...",NaN,NaN,NaN,NaN,NaN,"[consent] = ""0""",NaN,NaN,NaN
gender,registration_visit,NaN,radio,Gender,"0, Female | 1, Male | 99, Undefined",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
race,registration_visit,NaN,radio,Race,"1, White or Caucasian | 2, Black or African Am...",Put other for anything not listed here (Middle...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
visitcategory,baseline_and_follow_up,NaN,radio,What type of visit is this?,"1, Baseline Visit | 2, Follow Up Visit",Please make sure this matches the event name a...,NaN,NaN,NaN,NaN,[registration_visit_arm_1][ibddiag] = '1' and ...,y,NaN,NaN
visittype,baseline_and_follow_up,NaN,radio,BV4. Visit Type,"1, Baseline Visit, within 3 months of initial ...",NaN,NaN,NaN,NaN,NaN,[visitcategory]='1',NaN,NaN,NaN


In [281]:
cb[cb['Field Type'] == 'checkbox'].head()

,Form Name,Section Header,Field Type,Field Label,"Choices, Calculations, OR Slider Labels",Field Note,Text Validation Type OR Show Slider Number,Text Validation Min,Text Validation Max,Identifier?,Branching Logic (Show field only if...),Required Field?,Custom Alignment,Matrix Group Name
Variable / Field Name,,,,,,,,,,,,,,
prior_protocol_number,registration_visit,NaN,checkbox,"If Yes, what other studies?**","5, Biorepository (P00000529) | 1, CHIMP (X06-1...",NaN,NaN,NaN,NaN,NaN,[otherstudies] = '1',NaN,NaN,NaN
sampletype,registration_visit,NaN,checkbox,What samples has the patient given?**,"1, Blood | 3, Fecal | 4, GI Biopsy | 5, Oral s...",NaN,NaN,NaN,NaN,NaN,[prior_protocol_number(5)] = '1' or [prior_pro...,NaN,NaN,NaN
historyfrom,baseline_and_follow_up,NaN,checkbox,BV10. History obtained from (choose all that a...,"1, Patient | 2, Parent or guardian | 3, Chart ...",NaN,NaN,NaN,NaN,NaN,[visitcategory]='1',NaN,LH,NaN
historyfrom2,baseline_and_follow_up,NaN,checkbox,FV11. History obtained from:,"1, Patient | 2, Parent or guardian | 3, Chart ...",NaN,NaN,NaN,NaN,NaN,[visitcategory]='2',NaN,LH,NaN
jewanctype,baseline_and_follow_up,NaN,checkbox,"If yes, indicate Jewish ancestry subtype (choo...","1, Sephardic | 2, Ashkenazi | 3, Unknown",NaN,NaN,NaN,NaN,NaN,"[jeworigin]=""1"" or [jeworiginnew]=""1""",NaN,LH,NaN


In [ ]:
d.historyfrom

In [13]:
di_ = di.reset_index()

In [14]:
di_.iloc[:,0].apply(lambda x: True if 'arm' in x else False).sum()

0

In [15]:
def clean_branching_logic(ser):
    ser.str

In [16]:
ser = di['Branching Logic (Show field only if...)']

In [17]:
rules = r"""
"""

In [18]:
blogic_unique = list(ser.dropna().str.replace('"',"'").unique())

# Parser grammer for branching logic

In [33]:
w = '[age] >= 18 or [ageatdiag] >= 18'
x = "([ibdtype] = '0' or [newidbdiag] = '0') and [ibdcondl(6)] = '1'"
y = "([baseline_visit_arm_1][enviquest] = '1' and [visitcategory] = '1') or [enviquest2] = '1'"
z= "[meddraae25] = '99' or [meddraae24] = '99' or [meddraae23] = '99' or [meddraae22] = '99' or [meddraae21] = '99' or [meddraae20] = '99' or [meddraae19] = '99' or [meddraae18] = '99' or [meddraae17] = '99' or [meddraae166] = '99' or [meddraae15] = '99' or [meddraae14] = '99' or [meddraae13] = '99' or [meddraae12] = '99' or [meddraae11] = '99' or [meddraae10] = '99' or [meddraae9] = '99' or [meddraae8] = '99' or [meddraae7] = '99' or [meddraae6] = '99' or [meddraae5] = '99' or [meddraae4] = '99' or [meddraae3] = '99' or [meddraae2] = '99' or [meddraae1] = '99' or [meddraae26] = '99' or [meddraae27] = '99' or [meddraae28] = '99' or [meddraae29] = '99' or [meddraae16] = '99' "

In [21]:
p.alphanums

'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789'

In [221]:
# misc
lbrack = p.Literal("[").suppress()
rbrack = p.Literal("]").suppress()
lparen = p.Literal("(").suppress()
rparen = p.Literal(")").suppress()
anyquote = p.oneOf("""" '""").suppress()
empty_quotes = p.Literal("''") | p.Literal('""')

# variables
checkbox_val = p.Optional('('+p.Word(p.nums)+')')
variable_simple = p.Combine(lbrack+p.Word(p.alphanums + "_")+checkbox_val+rbrack)
variable = p.Group(p.OneOrMore(variable_simple)).setResultsName('variable')

# operators
operator = p.oneOf("<> >= <= > < =")

# logic gates
AND = p.Literal("and")
OR = p.Literal("or")
logic_gate = p.Group(AND | OR).setResultsName('logic_gate')

# comparison vals
comparison_val = p.Group(p.Word(p.alphanums) | (anyquote + p.Word(p.alphanums) + anyquote) | empty_quotes).setResultsName('comparison_val')

# branch expression
# expression = 
expression = (variable) + logic_gate + (comparison_val)


# atom = expression | variable | comparison_val 
atom = p.Group(variable | comparison_val).setResultsName('atom')

def branch_parser():

    
    expr = p.operatorPrecedence( atom,
                                [(operator, 2, p.opAssoc.LEFT),
                                 (logic_gate, 2, p.opAssoc.RIGHT),]
                                )
    return expr

In [222]:
bparser = branch_parser()

In [231]:
pstring = y

print(pstring)

xparser = branch_parser()
# list(xparser.scanString(pstring))
out = xparser.parseString(pstring, parseAll=True)
print(out)

([baseline_visit_arm_1][enviquest] = '1' and [visitcategory] = '1') or [enviquest2] = '1'
[[[[[['baseline_visit_arm_1', 'enviquest']], '=', [['1']]], ['and'], [[['visitcategory']], '=', [['1']]]], ['or'], [[['enviquest2']], '=', [['1']]]]]


In [232]:
print(out.asXML())


<ITEM>
  <ITEM>
    <ITEM>
      <atom>
        <atom>
          <variable>
            <ITEM>baseline_visit_arm_1</ITEM>
            <ITEM>enviquest</ITEM>
          </variable>
        </atom>
        <ITEM>=</ITEM>
        <atom>
          <comparison_val>
            <ITEM>1</ITEM>
          </comparison_val>
        </atom>
      </atom>
      <logic_gate>
        <ITEM>and</ITEM>
      </logic_gate>
      <atom>
        <atom>
          <variable>
            <ITEM>visitcategory</ITEM>
          </variable>
        </atom>
        <ITEM>=</ITEM>
        <atom>
          <comparison_val>
            <ITEM>1</ITEM>
          </comparison_val>
        </atom>
      </atom>
    </ITEM>
    <logic_gate>
      <ITEM>or</ITEM>
    </logic_gate>
    <atom>
      <atom>
        <variable>
          <ITEM>enviquest2</ITEM>
        </variable>
      </atom>
      <ITEM>=</ITEM>
      <atom>
        <comparison_val>
          <ITEM>1</ITEM>
        </comparison_val>
      </atom>
    </atom

In [144]:
import xml.etree.ElementTree as ET
from lxml import etree
from lxml import objectify
import xmltodict

In [169]:
print(out.asList())

[[[[['ibdtype'], '=', ['0']], ['or'], [['newidbdiag'], '=', ['0']]], ['and'], [['ibdcondl(6)'], '=', ['1']]]]


In [121]:
et = ET.fromstring(out.asXML())

In [111]:
root.items()

[]

In [225]:
lx = etree.fromstring(out.asXML())

In [233]:
variables_lx = lx.xpath('//variable')
comparison_vals_lx = lx.xpath('//comparison_val')

In [244]:
[o.tag for o in variables_lx]

['variable', 'variable', 'variable']

In [235]:
obj = objectify.fromstring(out.asXML())

In [236]:
variables = obj.xpath('//variable')
comparison_vals = obj.xpath('//comparison_val')

In [257]:
for v in [o["ITEM"][:] for o in variables]:
    print(":".join([str(v_) for v_ in v]))

baseline_visit_arm_1:enviquest
visitcategory
enviquest2


In [248]:
[o.ITEM[:] for o in obj.xpath('//variable')]

[['baseline_visit_arm_1', 'enviquest'], ['visitcategory'], ['enviquest2']]

In [249]:
[o.ITEM[:] for o in obj.xpath('//comparison_val')]

[[1], [1], [1]]

In [154]:
res_dict = xmltodict.parse(out.asXML())

In [155]:
res = munchify(res_dict)

In [161]:
res.ITEM.ITEM.variable.ITEM

'='

In [90]:
res.getName()

In [332]:
parsed_logics = []

In [333]:
for l in blogic_unique:
    parsed_logics.append(bparser.parseString(l))

In [334]:
len(parsed_logics)

189

([([(['enviquest2'], {}), '=', '1'], {}), 'and', ([(['visitcategory'], {}), '=', '2'], {})], {})

In [344]:
blogic_unique

["[consent] = '0'",
 "[noconsent] = '4'",
 "[otherstudies] = '1'",
 "[prior_protocol_number(5)] = '1' or [prior_protocol_number(1)] = '1' or [prior_protocol_number(2)] = '1' or [prior_protocol_number(3)] = '1' or [prior_protocol_number(4)] = '1'",
 "[sampletype(8)] = '1'",
 "[registration_visit_arm_1][ibddiag] = '1' and [registration_visit_arm_1][consent] = '1'",
 "[registration_visit_arm_1][ibddiag] = '1'",
 "[visitcategory] = '1' or [visitcategory] = '2'",
 "[visitcategory]='1'",
 "[visitcategory]='2'",
 "[visitcategory]='2' and [ibddiagchange]='1'",
 "[ancestrynew]='1'",
 "[jeworigin]='1' or [jeworiginnew]='1'",
 "[visitcategory] = '1' or [famhisnew] = '1'",
 "([visitcategory] = '1' or [famhisnew] = '1') and [anyfamauto]='1'",
 "([visitcategory] = '1' or [famhisnew]='1') and [anyfamauto]='1'",
 "[famcd(8)] = '1'",
 "[famuc(8)] = '1'",
 "[famun(8)] = '1'",
 "([visitcategory] = '1' or [famhisnew]='1')",
 "([visitcategory] = '1' or [famhisnew]='1') and [famauto]='1'",
 "[famautotype(7)

# Convert to HTML report and move to reports folder

In [ ]:
# rv.reprt_dir

In [ ]:
# !jupyter nbconvert --to html_toc $rv.ipnb

In [ ]:
# !mkdir -p $rv.reprt_dir
## !mv -f $rv.html $rv.files $rv.reprt_dir